<a href="https://colab.research.google.com/github/Aashika100-jpg/AI-Lab/blob/main/A*.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq

# Class to represent the state of the puzzle
class PuzzleState:
    def __init__(self, board, parent=None, move="", g=0, h=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.g = g  # Cost from start node to this node
        self.h = h  # Heuristic cost from this node to goal
        self.f = g + h  # Total estimated cost

    def __lt__(self, other):
        return self.f < other.f

# Function to find the position of the blank (zero) tile
def find_blank(board):
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return i, j

# Generate possible moves: Up, Down, Left, Right
def get_neighbors(state):
    x, y = find_blank(state.board)
    directions = {
        "Up": (x - 1, y),
        "Down": (x + 1, y),
        "Left": (x, y - 1),
        "Right": (x, y + 1),
    }
    neighbors = []

    for move, (new_x, new_y) in directions.items():
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            new_board = [row[:] for row in state.board]
            new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
            neighbors.append(PuzzleState(new_board, state, move, state.g + 1, 0))

    return neighbors

# Manhattan distance heuristic function
def heuristic(board, goal):
    distance = 0
    for i in range(3):
        for j in range(3):
            if board[i][j] != 0:
                x, y = divmod(goal.index(board[i][j]), 3)
                distance += abs(x - i) + abs(y - j)
    return distance

# A* algorithm implementation
def a_star(start, goal):
    start_state = PuzzleState(start, None, "", 0, heuristic(start, sum(goal, [])))
    goal_flat = sum(goal, [])

    open_list = []
    closed_set = set()
    heapq.heappush(open_list, start_state)

    while open_list:
        current_state = heapq.heappop(open_list)

        if current_state.board == goal:
            path = []
            while current_state.parent:
                path.append(current_state.move)
                current_state = current_state.parent
            return path[::-1]

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in get_neighbors(current_state):
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue

            neighbor.h = heuristic(neighbor.board, goal_flat)
            neighbor.f = neighbor.g + neighbor.h

            heapq.heappush(open_list, neighbor)

    return None

# Function to print the puzzle board
def print_board(board):
    for row in board:
        print(row)
    print()

# Example usage
start_state = [
    [1, 2, 3],
    [0, 4, 6],
    [7, 5, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

print("Initial State:")
print_board(start_state)

solution = a_star(start_state, goal_state)

if solution:
    print("Solution steps:", solution)
else:
    print("No solution found.")
